# Importing and preparing supermarkets data

## Libraries and settings

In [62]:
# Libraries
import os
import fnmatch
import pandas as pd

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Get current working directory
print('Current working directory:', os.getcwd())

# Show .json files in the current working directory
flist = fnmatch.filter(os.listdir('.'), '*.json')
for i in flist:
    print(i)

Current working directory: /workspaces/data_analytics/Week_02
supermarkets.json


## Importing data

In [63]:
# Read the data to a pandas data frame
df1 = pd.read_json('supermarkets.json', encoding='utf-8')
df1.head(5)

,type,id,lat,lon,tags
0,node,33126515,47.155616,9.037915,"{'brand': 'Spar', 'brand:wikidata': 'Q610492',..."
1,node,36726161,47.226191,8.980329,"{'addr:city': 'Uznach', 'addr:housenumber': '2..."
2,node,39768209,47.225069,8.969981,"{'addr:city': 'Uznach', 'addr:postcode': '8730..."
3,node,39947904,47.376732,8.542161,"{'addr:city': 'Zürich', 'addr:country': 'CH', ..."
4,node,48932835,47.375020,8.522895,"{'addr:city': 'Zürich', 'addr:housenumber': '7..."


## Count number of rows and columns in the data frame

In [64]:
# Dimension (rows, columns)
print('Dimension:', df1.shape)

# Number of rows
print('Number of rows:', df1.shape[0])

# Number of columns
print('Number of columns:', df1.shape[1])

Dimension: (3392, 5)
Number of rows: 3392
Number of columns: 5


## Column 'tags' is a pandas Series with dictionaries -> change to data frame

In [65]:
# Type of the first item of column tags
print(type(df1.tags))
print(type(df1.tags[0]))

# Content of the first item of column tags
print(df1.tags[0].keys())

# Change to data frame
df2 = pd.DataFrame.from_records(df1.tags)
df2[['brand', 'shop', 'addr:city', 'addr:street', 'addr:housenumber', 'addr:postcode']]

# Rename selected columns
df2 = df2.rename(columns={'addr:city': 'city',
                          'addr:street':'street',
                          'addr:housenumber': 'housenumber',
                          'addr:postcode': 'postcode'})

# Show first records of data frame
df2.head()

<class 'pandas.core.series.Series'>
<class 'dict'>
dict_keys(['brand', 'brand:wikidata', 'brand:wikipedia', 'name', 'opening_hours', 'shop'])


,brand,brand:wikidata,brand:wikipedia,name,opening_hours,shop,city,housenumber,postcode,street,...,source:addr,opening_date,postid,operator:website,diet:local,payment:account_cards,diet:fish,diet:mediterranean,diet:organic,diet:seafood
0,Spar,Q610492,en:SPAR (retailer),Spar,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00,supermarket,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Migros,Q680727,de:Migros,Migros,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17...",supermarket,Uznach,25,8730,Zürcherstrasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Coop,Q432564,NaN,Coop,NaN,supermarket,Uznach,NaN,8730,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Coop,Q432564,de:Coop (Schweiz),Coop,Mo-Sa 06:00-22:00,supermarket,Zürich,1,8001,Bahnhofbrücke,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Migros,Q680727,NaN,Migros,Mo-Sa 08:00-21:00; PH off,supermarket,Zürich,7,8004,Wengistrasse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Merge df1 and df2

In [66]:
# Merge df and df2
df = pd.merge(df1[['type', 'id', 'lat', 'lon']], 
              df2[['brand', 'shop', 'city', 'street', 'housenumber', 'postcode']],
              left_index=True, 
              right_index=True)
df.head(5)

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
2,node,39768209,47.225069,8.969981,Coop,supermarket,Uznach,NaN,NaN,8730
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004


## Count and identify the number of missing values (if any)

In [67]:
# Count missing values
print(pd.isna(df).sum())

# Identify rows with missing values, e.g.:
df.loc[pd.isna(df['city'])]

type              0
id                0
lat               0
lon               0
brand          1065
shop              0
city           1777
street         1608
housenumber    1680
postcode       1709
dtype: int64


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN
5,node,60271452,47.406671,9.305450,NaN,supermarket,NaN,NaN,NaN,NaN
6,node,70656485,47.491253,8.733981,NaN,supermarket,NaN,NaN,NaN,NaN
10,node,81321513,47.532917,9.066408,Landi,supermarket,NaN,NaN,NaN,NaN
13,node,95582038,47.050385,9.059214,NaN,supermarket,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3384,node,11083317088,46.862184,9.531169,Lidl,supermarket,NaN,NaN,NaN,NaN
3386,node,11098091830,46.205111,6.130174,Coop,supermarket,NaN,NaN,NaN,NaN
3387,node,11099817248,46.928691,7.561873,NaN,supermarket,NaN,NaN,NaN,NaN
3388,node,11103235832,46.166742,8.771970,Migros,supermarket,NaN,NaN,NaN,NaN


## Count and identify duplicated values (if any)

In [68]:
# Count duplicated values
print(df.duplicated().sum())

# Identify rows with duplicated values, e.g.:
df[df[['id']].duplicated()]

0


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode


## Get data types of all variables

In [69]:
# Get data types (note that in pandas, a string is referred to as 'object')
df.dtypes

type            object
id               int64
lat            float64
lon            float64
brand           object
shop            object
city            object
street          object
housenumber     object
postcode        object
dtype: object

### Save data to file

In [70]:
df.to_csv('supermarkets_data_prepared.csv', 
          sep=",", 
          encoding='utf-8',
          index=False)

### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [71]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 5.15.0-1041-azure
Datetime: 2023-09-28 19:10:47
Python Version: 3.10.13
-----------------------------------


## Additional filters on supermarkets

### Task 3f: Filter only Migros supermarkets in the city of Zürich

In [72]:
# Identify rows with missing values, e.g.:
df_filtered_3f = df.loc[(df['city'] == 'Zürich')]
df_filtered_3f

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004
8,node,75749133,47.340967,8.530601,ALDI,supermarket,Zürich,Albisstrasse,81,8038
9,node,79977755,47.340070,8.530546,Coop,supermarket,Zürich,Alte Kalchbühlstrasse,15,8038
11,node,83330862,47.344749,8.529981,Migros,supermarket,Zürich,Etzelstrasse,3,8038
...,...,...,...,...,...,...,...,...,...,...
3007,node,8541239873,47.387455,8.517340,Coop,supermarket,Zürich,Maschinenstrasse,10,8005
3286,node,10235419981,47.371434,8.543643,NaN,supermarket,Zürich,Ankengasse,7,8001
3331,node,10678973133,47.391476,8.515767,NaN,supermarket,Zürich,Förrlibuckstrasse,NaN,NaN
3338,node,10711459945,47.369962,8.509175,Coop,supermarket,Zürich,Birmensdorferstrasse,310,8055


### Task 3g: Filter and count all Coop supermarkets in the cities of Zürich, Basel & Bern

In [73]:
df_filtered_3g = df.loc[(df['brand'] == 'Coop') & (
    (df['city'] == 'Zürich') | (df['city'] == 'Basel') | (df['city'] == 'Bern'))]
df_filtered_3g


,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
9,node,79977755,47.340070,8.530546,Coop,supermarket,Zürich,Alte Kalchbühlstrasse,15,8038
59,node,265776668,47.376417,8.559594,Coop,supermarket,Zürich,Zürichbergstrasse,75,8044
63,node,266630559,47.377716,8.511219,Coop,supermarket,Zürich,Badenerstrasse,333,8003
70,node,267345511,47.385809,8.516574,Coop,supermarket,Zürich,Maagplatz,1,8005
72,node,267468996,47.364872,8.521006,Coop,supermarket,Zürich,Uetlibergstrasse,20,8045
75,node,268603429,47.367360,8.546174,Coop,supermarket,Zürich,Theaterstrasse,18,8001
81,node,270692983,47.357940,8.554646,Coop,supermarket,Zürich,Seefeldstrasse,123,8008
84,node,271028686,47.366773,8.548079,Coop,supermarket,Zürich,Stadelhoferstrasse,10,8001
86,node,271029581,47.364300,8.555129,Coop,supermarket,Zürich,Forchstrasse,4,8008


### Task 3h: filter supermarkets with available brand, city, house number and post code

In [74]:
df_filtered_3h = df.loc[
    (pd.notnull(df['brand'])) &
    (pd.notnull(df['city'])) &
    (pd.notnull(df['housenumber'])) &
    (pd.notnull(df['postcode']))
]

df_filtered_3h

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406
8,node,75749133,47.340967,8.530601,ALDI,supermarket,Zürich,Albisstrasse,81,8038
...,...,...,...,...,...,...,...,...,...,...
3350,node,10814018169,47.353857,8.436716,Coop,supermarket,Birmensdorf (ZH),Zürcherstrasse,9,8903
3362,node,10982669725,47.349782,8.258690,ALDI,supermarket,Villmergen,Grenzweg,3,5612
3370,node,11025130806,47.059301,7.620697,Denner,supermarket,Burgdorf,Lyssachstrasse,27,3400
3378,node,11049758254,47.338327,8.520261,Spar,supermarket,Zürich,Spinnereiplatz,4,8041


### Task 3i: Include opening hours as additional variable in the data frame

In [75]:
df3i = pd.merge(df1[['type', 'id', 'lat', 'lon']], 
              df2[['brand', 'shop', 'city', 'street', 'housenumber', 'postcode', 'opening_hours']],
              left_index=True, 
              right_index=True)
df3i.head(5)

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
2,node,39768209,47.225069,8.969981,Coop,supermarket,Uznach,NaN,NaN,8730,NaN
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off


### Task 3j: Filter supermarkets with available opening hours.

In [76]:
df_filtered_3j = df3i.loc[
    (pd.notnull(df3i['opening_hours']))
]

df_filtered_3j

,type,id,lat,lon,brand,shop,city,street,housenumber,postcode,opening_hours
0,node,33126515,47.155616,9.037915,Spar,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 08:00-17:00
1,node,36726161,47.226191,8.980329,Migros,supermarket,Uznach,Zürcherstrasse,25,8730,"Mo-Th 08:00-19:00, Fr 08:00-20:00, Sa 07:30-17..."
3,node,39947904,47.376732,8.542161,Coop,supermarket,Zürich,Bahnhofbrücke,1,8001,Mo-Sa 06:00-22:00
4,node,48932835,47.375020,8.522895,Migros,supermarket,Zürich,Wengistrasse,7,8004,Mo-Sa 08:00-21:00; PH off
7,node,70656488,47.491874,8.706448,Migros,supermarket,Winterthur,Zürcherstrasse,102,8406,Mo-Fr 07:30-20:00; PH off; Sa 08:00-19:00
...,...,...,...,...,...,...,...,...,...,...,...
3378,node,11049758254,47.338327,8.520261,Spar,supermarket,Zürich,Spinnereiplatz,4,8041,Mo-Sa 07:00 - 22:00
3380,node,11055427937,47.134219,8.045801,Denner,supermarket,NaN,Am Heubächli,1,NaN,off; 2023 Oct 5+ unknown
3387,node,11099817248,46.928691,7.561873,NaN,supermarket,NaN,NaN,NaN,NaN,Mo-Th 08:00-19:00; Fr 08:00-20:00; Sa 07:30-17:00
3388,node,11103235832,46.166742,8.771970,Migros,supermarket,NaN,NaN,NaN,NaN,Mo-Fr 07:30-19:00; Th 07:30-20:00; Sa 07:30-18:30


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [77]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 5.15.0-1041-azure
Datetime: 2023-09-28 19:16:11
Python Version: 3.10.13
-----------------------------------
